In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sax
import jax.numpy as jnp
from jax import config
config.update("jax_enable_x64", True)

from simphony.libraries import ideal
from simphony.utils import dict_to_matrix
from simphony.time_domain.utils import gaussian_pulse
from simphony.time_domain.pole_residue_model import IIRModelBaseband
from simphony.time_domain.utils import pole_residue_to_time_system


ImportError: cannot import name 'pole_residue_to_time_system' from 'simphony.time_domain.utils' (c:\Users\Owner\time_domain_work\simphony\simphony\time_domain\utils.py)

In [ ]:

netlist = {
    "instances": {
        "wg": "waveguide",
        "hr": "half_ring",
    },
    "connections": {
        "hr,o2": "wg,o0",
        "hr,o3": "wg,o1",
    },
    "ports": {
        "o0": "hr,o0",
        "o1": "hr,o1",
    }
}

circuit, info = sax.circuit(
    netlist=netlist,
    models={
        "waveguide": ideal.waveguide,
        "half_ring": ideal.coupler,
    }
)


In [ ]:

num_measurements = 200
model_order = 50
center_wvl = 1.548  # Center wavelength (µm)
wvl = np.linspace(1.5, 1.6, num_measurements)  # Wavelength range (µm)

# Perform simulation
s = circuit(wl=wvl, wg={"length": 77.0, "loss": 100})
S = np.asarray(dict_to_matrix(s))  # Convert the result to a matrix
model = IIRModelBaseband(wvl, center_wvl, S, model_order)


In [ ]:

N = int(1000)  # Number of time steps
T = 4e-11      # Total time duration (40 ps)
t = jnp.linspace(0, T, N)  # Time array
t0 = T/2 - 5e-12  # Pulse start time
std = 1e-12       # Pulse standard deviation

# Define input signals
impulse_pass = {
    'o0': gaussian_pulse(t, t0 - 0.5 * t0, std),
    'o1': jnp.zeros_like(t)
}

# Convert frequency domain model to a time-domain system
tsys = pole_residue_to_time_system(model)


In [ ]:

# Simulate the system's response to the input signals
outputs = tsys.response(impulse_pass)

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # 2 rows, 2 columns

# Plot input signals
for i in range(2):
    axs[i, 0].plot(t, jnp.abs(impulse_pass[f'o{i}'])**2)
    axs[i, 0].set_title(f'Input Signal {i+1}')
    axs[i, 0].set_xlabel('Time (s)')
    axs[i, 0].set_ylabel('Intensity')

# Plot output signals
for i in range(2):
    axs[i, 1].plot(t, jnp.abs(outputs[f'o{i}'])**2)
    axs[i, 1].set_title(f'Output Signal {i+1}')
    axs[i, 1].set_xlabel('Time (s)')
    axs[i, 1].set_ylabel('Intensity')

# Adjust layout
plt.tight_layout()
plt.show()
